**Reference:** https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b  
**Reference (card images):** https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10  

Part A: Dog Detector

Part B: Card Detector **(This is covered in the video, please do not not do this part and only do parts A and C)**

Part C: Custom Object Detector

# BEFORE YOU BEGIN:
Make sure that inside the "Colab Notebooks" folder inside your Google Drive that you create a folder called "ObjDetection-UserGeneratedModel", and put this colab script inside this folder. The commands below depend on this file structure.

When you are ready to move on to part C, rename your folder for part A to something like ObjectDetection-Dog, and create a new folder for part C with the same name "ObjDetection-UserGeneratedModel".

Also note that the default files from the materials folder are for part A and do not need to be modified. In the video, you will notice that the files are modified to have the correct output for part B, make sure you also make the necessary edits for part C. To clarify, **you do not need to do part B,** so you will not be using the card images and annotations in the materials link.

#**Step 1:**   
Run with TensorFlow 1.x  
Select GPU under menu Runtime - Change runtime type.  
Activate GPU for hardware acceleration used in training.


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

#**Step 2:**  
Mount your Google drive so can access folders/files.
Copy/paste the authorization key into the box to allow access.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

#**Step 3:**  
Change directory to your working folder and clone the TensorFlow models repository.  
Will create a folder  'models' --> community, official, research, etc.  
We'll be working mostly in the the models-->research folder and subs under it.  
Once this step is complete, you definitely will not need to re-run it even if your session expires.



In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/
!git clone https://github.com/tensorflow/models.git


#**Step 4:**  
Install needed tools:  
protobuf-compiler, python-pil, python-lxml, python-tk, Cython  

Also install the current version of tf-slim (1.1)  
Do it inside the object_detection/builders directory so can find the tensorflow.contrib module during the model builder test step.


In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/builders
!pip install --upgrade tf-slim


#**Step 5:**  
Compile the model definition.  
Need to be in the /models/research/ folder.

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/
!protoc object_detection/protos/*.proto --python_out=.

#**Step 6:**  
Set the environment.


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Colab Notebooks/ObjDetection-UserGeneratedModel/models/research/:/content/drive/Colab Notebooks/ObjDetection-UserGeneratedModel/models/research/slim'

#**Step 7:**  
**Run this before every session to setup the environment.**

In [ ]:
#####ALWAYS NEED TO RERUN THIS ON EVERY SESSION RESTART!!!
#Run in the slim directory
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/slim/
!python setup.py build
!python setup.py install

#Now run it again in the research directory
%cd ../

!python setup.py build
!python setup.py install

#**Step 8:**  
Run a test script to confirm all the training tools have been installed and the environment is setup properly. 

This script should run and output no errors if the environment is setup.

Need to be in the /object_detection/builders directory.


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/builders/
!python model_builder_test.py

#**Step 9:**  
Setup the annotated images files for the training and testing.  

**Make sure to check the resolution of your images.  Dog example was 640 x 480  RGB**   
**Used higher res in the cards example, but need to make sure the annotation is on the resized image!!**  


Create 'images' folder under object_detection and subfolders 'test' and 'train' under 'images' (case sensitive).  
Copy your raw jpg images and annotation xml files into these folders, e.g. file_001-040.jpg and .xml into train and file_041-050.jpg and .xml into test.  **You will want approximately 20% of the images and annotations in the test folder and 80% in the train folder.**

**For Part C: Read these steps carefully so that you won't have to repeat any steps.** 

1) Find a common household object and take around 55 pictures of it using your phone with varying backgrounds and distances and with multiple in the same picture (if possible). Try to take these pictures landscape. The first 50 pictures will be annotated and used for the test and train folders for this step similarly to part A. The last 5 pictures can be used for Step 19 when you actually test your model, so feel free not to annotate those 5.

2) Resize these pictures to 640x480. See below for how to resize. This will speed up the training process since the pictures taken by your phone will be too high quality and the script actually will not work if you do not resize the images first. If some of your pictures are taken portrait style, rotate the images 90 degrees so that they are horizontal **before resizing**.

Use the following script to resize if needed.  
import glob  
from PIL import Image  
import os  
class_name = 'CLASSNAME'  
os.mkdir('resized')  
file_list = sorted(glob.glob('*.jpg'))  
for idx,file_name in enumerate(file_list):  
  im = Image.open(file_name)  
  new_width  = 640  
  new_height = 480  
  im = im.resize((new_width, new_height), Image.ANTIALIAS)  
  im.save('resized/' + class_name + '_' + str(idx+1).zfill(3) + '.jpg')

**Alternatively, use the following link:** https://bulkresizephotos.com/en

3) Use the annotator under the ML110 materials link to annotate the resized images. Don't use the "default" label, make sure to create a label with the name of the object and use that label to label the objects. 

4) Now that you have both the images and annotations, follow the same steps for part A. 

Run the following script to convert between xml and csv.  
The output csv files will be stored in /object_detection/data  called test_labels.csv and train_labels.csv. After running this step, locate these files and open them up to make sure that you have done this step properly.


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/


import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
def xml_to_csv(path):
    xml_list = []
    print('path=', path)
    for xml_file in glob.glob(path + '/*.xml'):
        print('xml_file=', xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            print('member=', member)
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(float(member[4][0].text)),
                     int(float(member[4][1].text)),
                     int(float(member[4][2].text)),
                     int(float(member[4][3].text))
                     )
            xml_list.append(value)
    print('xml_list= ', xml_list)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
def main(directory_list):
    for Image_cat in directory_list:
        image_path = os.path.join(os.getcwd(), 'images/{}'.format(Image_cat))
        print('image_path= ', image_path)
        xml_df = xml_to_csv(image_path) 
        xml_df.to_csv('data/{}_labels.csv'.format(Image_cat), index=None)
        print('Successfully converted xml to csv.')
main(['train','test'])

#**Step 10:**  
Upload the "generate_tfrecord.py" file from the materials folder to the /models/research/object_detection directory.  All the files are already setup for part A and part C, so no further modifications to the file are needed. In the video, the correct file is shown for part B, the card example.

Now generate the training and testing record files.
Run the script each for training and testing.  

python generate_tfrecord.py --label='LABEL' --csv_input=data/train_labels.csv --output_path=data/train.record --img_path=images/train  
python generate_tfrecord.py --label='LABEL' --csv_input=data/test_labels.csv --output_path=data/test.record --img_path=images/test  

Where LABEL is the label from the csv file (class column in the file). e.g. LABEL=='dog' in the dog example.  

If this step was run correctly, you should have two new files under data: "test.record" and "train.record".

**For part C:**

Modify  the label command line arguments
--label1='LABEL1' with the name of your custom object.

Need to be in the object_detection directory.  


If get error ModuleNotFoundError: No module named 'object_detection' it's likely because the session expired and was restarted.  In that case need to rerun step 7. 


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/
#### ****
#### NOTE NEED TO CHANGE LABEL TO EACH OF YOUR LABELS IN THE FILES: train_labels.csv and test_labels.csv
#### ****
#!python generate_tfrecord.py --label='LABEL' --csv_input=data/train_labels.csv --output_path=data/train.record --img_path=images/train
#!python generate_tfrecord.py --label='LABEL' --csv_input=data/test_labels.csv --output_path=data/test.record --img_path=images/test

!python generate_tfrecord.py --label1='dog' --csv_input=data/train_labels.csv --output_path=data/train.record --img_path=images/train
!python generate_tfrecord.py --label1='dog' --csv_input=data/test_labels.csv --output_path=data/test.record --img_path=images/test

#**Step 11:**  
Get the pre-trained object detection model from TensorFlow and decompress it.    
Need to be in the /object_detection/ directory.  Both the tar file and the extracted directory will be stored there.   

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/

!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
!tar -xvf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

#**Step 12:**  
Create a directory under object_detection called 'training'.  
Upload the ssd_mobilenet_v1_coco.config file from the materials folder and store it in /training.  

For Part B this file was modified because there was more than 1 class in the model, but for Part C you should only have 1 object in your model so no modifications are necessary for both Part A and Part C.


#**Step 13:**
Upload file 'object-detection.pbtxt' from the materials folder into the training directory.

This file contains records of a mapping of the id's to the labels.  
If you have more than one object you are training then need to have more than one record.  
Follow the id mapping you assigned in the generate_tfrecord step.  
Need to save to /object_detection/training/ directory.  
At this point the training folder should only contain two files, in the video there appears to be many more files and these will be added in the next step.
Modifications are only needed in part C, the video shows the correct modifications needed for part B.




#**Step 14:**  
Execute training.  (This is a long step, usually takes around 30 minutes.)

Copy or move the train.py file from the 'legacy' folder into the object detection folder (move it up one level)
This training will run indefinitely.  You can run for a fixed number of steps by adding num_steps in the ssd_mobilenet_v1_coco.config file. (just search for num_steps and uncomment the line).  
Otherwise, stop the cell when loss is < 1.0. Don't worry if the training somehow gets interrupted or if you accidentally stop this block too early, the training will continue on the last model saved (the script saves a model every 1000 steps or so)

Need to be in object_detection directory to execute.

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/

!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_coco.config

#**Step 15:**  
Export the inference graph.  
Create a folder called trained_inference_graph under the object_detection directory then run this script from the object_detection directory.  

**Update model.ckpt-xxxx in the python command line to the highest number .ckpt file stored in the /training directory.**

If get error ModuleNotFoundError: No module named 'object_detection' it's likely because the session expired and was restarted.  In that case need to rerun step 7.  
Also if the session expired, make sure you have tensorflow 1.x active (step 1)


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/

!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_coco.config --trained_checkpoint_prefix training/model.ckpt-XXXX --output_directory trained_inference_graph/

#**Step 16:**  
Zip the inference graph.  
Name the graph according to your object detection project.

In [ ]:
#### Edit PROJECT to your object detection project.
!zip -r PROJECT.zip trained_inference_graph

#**Step 17:**  
#Reinstall tf-slim (to 1.2) here...
You'll have to install tf-slim-1.2 (was previously tf-slim-1.1.0)  
!pip install git+https://github.com/google-research/tf-slim.git


In [ ]:
!pip install git+https://github.com/google-research/tf-slim.git


#**Step 18:**  
Find the folder called 'test_images' under object_detection.  
Copy some images to test in the test_images folder and rename image1.jpg, image2.jpg, etc.  

#**Step 19:**  
Test the model by uploading images into the test_images folder, naming them image1.jpg, image2.jpg, etc.

Update the number of images to test in the code by modifying the range for the variable TEST_IMAGE_PATHS

Need to be in the /object_detection/ directory.  
You should see the images output with bounding boxes.  

If you've restarted your session:  
If you receive tf-slim specific errors (e.g. can't find tf-slim), run step 17.  
If you receive tensorflow specific errors (e.g. tensorflow has no module GraphDef) run step 1 (note, may also have to restart runtime from the main menu if get message that tensorflow is already loaded at version 2.x).

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



### Model preparation variable
MODEL_NAME = 'trained_inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training/object-detection.pbtxt'
NUM_CLASSES = 1 #remember number of objects you are training? cool.


### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)




### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




### Path to the test images folder
### UPDATE THE RANGE TO THE NUMBER OF IMAGES IN THE FOLDER
PATH_TO_TEST_IMAGES_DIR = 'test_images/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 10) ]
IMAGE_SIZE = (12, 8)




### Function to run inference on a single image which will later be used in an iteration
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict



### To iterate on each image in the test image path defined 
### NB define the range of numbers and let it match the number of imAGES IN TEST FOLDER +1
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=1)
  display(Image.fromarray(image_np))

#**Step 20**  

Next, we'll need to convert the TensorFlow model to TFLite so we can run it on the Pi.

Make a new directory called TFLite_model and create the TFLite inference graph. Update model.ckpt-xxxx in the python command line to the highest number .ckpt file stored in the /train or /training directory, depending on where the model is stored. 

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/

!mkdir TFLite_model
!python export_tflite_ssd_graph.py --pipeline_config_path=training/ssd_mobilenet_v1_coco.config --trained_checkpoint_prefix=training/model.ckpt-XXXX --output_directory=TFLite_model --add_postprocessing_op=true

#**Step 21**  
Install TensorFlow Lite Optimizing Converter (TOCO), and run the model through TOCO to get a model that can be used by TensorFlow Lite.  

Make sure that your graph def file points to the **tflite_graph.pb** file, **not** the frozen_inference_graph.pb file from the previously exported model.


In [ ]:
!pip install toco
%cd /content/drive/My\ Drive/Colab\ Notebooks/ObjDetection-UserGeneratedModel/models/research/object_detection/
!toco --graph_def_file=TFLite_model/tflite_graph.pb --output_file=TFLite_model/detect.tflite --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE --input_shape=1,300,300,3 --input_array=normalized_input_image_tensor --output_array='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --input_type=FLOAT --allow_custom_ops

#**Step 22:**
Create a TFLite compatible label map

TFLite's labelmaps are organized differently than that of TensorFlow. Instead of explicitly stating the name and ID, TFLite just lists each class.

Thus, we need to create a new label map that matches the TensorFlow Lite style. Open a text editor and list each class in order of their class number.  Because parts A and C only have 1 class trained on, simply type in the name of your class.
Then, save the file as “labelmap.txt” in the TFLite_model folder. 

#**Step 23:**
Move the TFLite_model folder over to your Pi and run the model

Download and zip your TFLite_model folder to your computer. 


As an alternative to copying your files over via thumbdrive, you can use SCP to copy it over.  

'scp myfile.txt pi@192.168.1.3:tflite1/'

This should prompt you to enter the Pi's password. Enter it, then it should show a progress percentage.

Access your Pi (use VNC viewer if you are operating your Pi remotely), navigate to the tflite1 folder.

Run `source tflite1-env/bin/activate` to activate the computer vision environment, and then run `python3 TFLite_detection_webcam.py --modeldir=TFLite_model/` to test your model.
